# Section: Initialization

- setting values for global variables
- loading required packages
- reading files
- setting up logging

In [1]:
# This code is for supervised learning using previous state to predict current action
# Code from gym-flight to be reused as much as possible
import tensorflow as tf
import sys
import os
sys.path.append('../gym-flight/')
import gym_flight
from gym_flight.utils.numpy_util import Sparse3dArray
import numpy as np
import math
import pandas as pd
from continuous import *
from full3d_util import get_last_ts_data, create_space_ts, get_range_df, get_action_continuous, get_env_action_aircraft, get_X_Y, fix_XY, unlist, bind, train_test_split
import multiprocessing as mp
import pickle
splits = mp.cpu_count() - 1

In [2]:
import logging
logging.basicConfig(
    filename='log.txt',
    level=logging.DEBUG,
    format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
x_length = 100
y_length = 100
z_length = 100
ts_range = 5
time_dim = 1
state_dim = [x_length, y_length, z_length]
learning_rate = 0.025
flight_data_path = "../gym-flight/data/processed_jfk.csv"
dtype_dict = {"id": str, "ts": np.int16, "lat": np.float32, "lon": np.float32, "altitude": np.float32, "speed": np.float32, "x": np.int16, "y": np.int16, "z": np.int16, "is_landing": np.int8}
flight_data = pd.read_csv(flight_data_path, dtype = dtype_dict)

# Discrete 100 * 100 * 100 space for each timestamp

In [3]:
def get_range_df_with_params(rng):
    return get_range_df(rng, flight_data, x_length, y_length, z_length, ts_range)

# Space for a timestamp remains the same. Only the action is continuous
if os.path.isfile('ts_full3d_space.bin'):
    space_X = pickle.load(open('ts_full3d_space.bin', 'rb'))
else:
    p = mp.Pool(processes = splits)
    # Arranging in reverse order to speed up computation
    rng = list(reversed(range(min(flight_data['ts']), max(flight_data['ts']) + 1)))
    split_rng = np.array_split(rng, splits)
    pool_results = p.map(get_range_df_with_params, split_rng)
    p.close()
    p.join()
    space_X = np.concatenate(pool_results, axis = 0)
    space_X = np.flip(space_X, axis = 0)
    pickle.dump(space_X, open("ts_full3d_space.bin", "wb"))


## Action

Coding action as set of 3 continuous variables:

- Change in ground speed
- Change in altitude
- Change in heading (azimuth)

In [4]:
if os.path.isfile('Y_train_full3d_space_continuous_action.bin'):
    X_train = pickle.load(open('X_train_full3d_space_multiclass_action.bin', 'rb'))
    Y_train = pickle.load(open('Y_train_full3d_space_continuous_action.bin', 'rb'))
    X_test = pickle.load(open('X_test_full3d_space_multiclass_action.bin', 'rb'))
    Y_test = pickle.load(open('Y_test_full3d_space_continuous_action.bin', 'rb'))
else:
    uniq_id = flight_data['id'].unique()
    #     uniq_id = uniq_id[:50]
    XY = get_X_Y(uniq_id, flight_data, x_length, y_length, z_length, space_X, discrete_action = False)
    X, Y = fix_XY(XY)
    X_train, Y_train, X_test, Y_test = train_test_split(X, Y)
    pickle.dump(X_train, open("X_train_full3d_space_multiclass_action.bin", "wb"))
    pickle.dump(Y_train, open("Y_train_full3d_space_continuous_action.bin", "wb"))
    pickle.dump(X_test, open("X_test_full3d_space_multiclass_action.bin", "wb"))
    pickle.dump(Y_test, open("Y_test_full3d_space_continuous_action.bin", "wb"))

train_ac_index = [len(X[0]) for X in X_train]
test_ac_index = [len(X[0]) for X in X_test]
X_train = unlist(unlist(X_train))
Y_train = bind(unlist(Y_train))
Y_train = Y_train.astype(np.float16)

X_test = unlist(unlist(X_test))
Y_test = bind(unlist(Y_test))
Y_test = Y_test.astype(np.float16)

Restricting data to 1500 ft/min descent rate

In [5]:
X_train = [X_train[i] for i, y in enumerate(Y_train) if (y[1] >= -1500 and y[1] <= 1500)]
Y_train = Y_train[(Y_train[:, 1] >= -1500) & (Y_train[:, 1] <= 1500), :]
print(len(X_train))
print(Y_train.shape)

X_test = [X_test[i] for i, y in enumerate(Y_test) if (y[1] >= -1500 and y[1] <= 1500)]
Y_test = Y_test[(Y_test[:, 1] >= -1500) & (Y_test[:, 1] <= 1500), :]
print(len(X_test))
print(Y_test.shape)

494709
(494709, 3)
203621
(203621, 3)


Getting minimum and maxmimum values for normalization

In [6]:
train_min = np.min(Y_train, axis = 0)
test_min = np.min(Y_test, axis = 0)
min_array = np.array([min(train_min[i], test_min[i]) for i in range(3)])
train_min = np.min(Y_train, axis = 0)
test_min = np.min(Y_test, axis = 0)
min_array = np.array([min(train_min[i], test_min[i]) for i in range(3)])

train_max = np.max(Y_train, axis = 0)
test_max = np.max(Y_test, axis = 0)
max_array = np.array([max(train_max[i], test_max[i]) for i in range(3)])
train_max = np.max(Y_train, axis = 0)
test_max = np.max(Y_test, axis = 0)
max_array = np.array([max(train_max[i], test_max[i]) for i in range(3)])

In [7]:
Y_train = (Y_train-min_array)/(max_array - min_array)
Y_test = (Y_test-min_array)/(max_array - min_array)

# Section on CNN, prediction, training and evaluation

In [8]:
state_dim = [x_length, y_length, z_length]
cnn = CNN_continuous(state_dim, learning_rate)

Instructions for updating:
Use keras.layers.conv3d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


# Running the model

In [9]:
saver = tf.train.Saver()

# unique_ts = flight_data['ts'].unique()
# for i in range(len(unique_ts)):
#     train_X.append(create_space_ts(ts = unique_ts.iloc[i], flight_data = flight_data, x_length = x_length, y_length = y_length, z_length = z_length, ts_range = ts_range))

# Row binding all the arrays
# train_X = np.concatenate(train_X, axis = 0)
perc = 0
epochs = 1
batch_size = 128
num_batches = len(X_train)//batch_size

In [10]:
completed = 0
epoch_train_losses = []
epoch_test_losses = []
ended = False
epoch = 0
sess = tf.Session()
# Initialize variables
sess.run(tf.global_variables_initializer())
while (epoch < epochs) and not(ended):
    completed = int(epoch * 100/epochs)
    if completed >= perc:
        logging.info(str(perc) + " % completed")
        perc = int(epoch * 100/epochs)

    batch_completed = 0
    mini_batch = 1
    mini_batch_train_losses = []
    mini_batch_test_losses = []
    while batch_completed < (len(X_train) - batch_size):
        train_X = X_train[batch_completed:(batch_completed + batch_size)]
        train_Y = Y_train[batch_completed:(batch_completed + batch_size)]
        loss = train(sess, cnn, train_X, train_Y)
        # Runs out of memory while evaluating on the complete test set. Only batch_size used for evaluating
        # This step should be randomized
        test_loss = get_loss(sess, cnn, X_test[0:batch_size], Y_test[0:batch_size])
        print('Epoch: {}/{}: '.format(epoch+1, epochs), 'mini-batch {}/{}: '.format(mini_batch, num_batches), "Train loss: {} ".format(loss), "Test loss: {} ".format(test_loss))
        batch_completed += batch_size
        mini_batch += 1
        mini_batch_train_losses.append(loss)
        mini_batch_test_losses.append(test_loss)
    epoch_train_losses.append(sum(mini_batch_train_losses)/len(mini_batch_train_losses))
    epoch_test_losses.append(sum(mini_batch_test_losses)/len(mini_batch_test_losses))
    # Early stopping check (callback should be used instead):
    if (epoch > 0) and (epoch_test_losses[epoch] > epoch_test_losses[epoch - 1]):
        saver.save(sess, "supervised_CNN_full3d_continuous")
        ended = True

    epoch += 1

Epoch: 1/1:  mini-batch 1/3864:  Train loss: 0.00522669916972518  Test loss: 745179.6875 
Epoch: 1/1:  mini-batch 2/3864:  Train loss: 753682.6875  Test loss: 0.9753214716911316 
Epoch: 1/1:  mini-batch 3/3864:  Train loss: 1.0503935813903809  Test loss: 2.972888946533203 
Epoch: 1/1:  mini-batch 4/3864:  Train loss: 3.1596667766571045  Test loss: 0.7117863297462463 
Epoch: 1/1:  mini-batch 5/3864:  Train loss: 0.5214526057243347  Test loss: 1.276354432106018 
Epoch: 1/1:  mini-batch 6/3864:  Train loss: 1.0747840404510498  Test loss: 0.08401143550872803 


KeyboardInterrupt: 

In [ ]:
saver.save(sess, "supervised_CNN_full3d_continuous")
Y_train.shape